## 一、載入套件與設定路徑

In [1]:
import pandas as pd
import numpy as np
import os
import warnings

# 忽略 pandas 的警告訊息
warnings.filterwarnings('ignore')

# --- 📂 Step 1: 設定檔案路徑 ---
# 根據您的指示，直接設定相對路徑
data_dir = "../DataSet/"
output_dir = "./" # 輸出到當前目錄

# 確保輸出目錄存在
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print(f"📂 資料讀取路徑設定為: {data_dir}")

📂 資料讀取路徑設定為: ../DataSet/


## 二、讀取原始資料

In [2]:
# --- 📥 Step 2: 讀取資料 ---
print("⏳ 正在讀取資料...")

try:
    # 1. 讀取交易紀錄
    txn_path = os.path.join(data_dir, 'acct_transaction.csv')
    df_txn = pd.read_csv(txn_path, dtype={
        'from_acct': str, 'to_acct': str, 
        'from_acct_type': str, 'to_acct_type': str,
        'txn_amt': float
    })

    # 2. 讀取警示帳戶 (標籤資料)
    alert_path = os.path.join(data_dir, 'acct_alert.csv')
    df_alert = pd.read_csv(alert_path, dtype={'acct': str})

    # 3. 讀取待預測帳戶 (提交目標)
    # 加入欄位名稱檢查機制，確保能正確讀取 ID
    predict_path = os.path.join(data_dir, 'acct_predict.csv')
    # 先讀取 header 確認欄位
    df_predict_raw = pd.read_csv(predict_path, nrows=0)
    predict_cols = df_predict_raw.columns.tolist()
    
    # 判斷 ID 欄位名稱
    if 'acct_test' in predict_cols:
        pid = 'acct_test'
    elif 'acct' in predict_cols:
        pid = 'acct'
    else:
        pid = predict_cols[0] # 預設第一個欄位
    
    df_predict = pd.read_csv(predict_path, dtype={pid: str})
    if pid != 'acct_test':
        df_predict = df_predict.rename(columns={pid: 'acct_test'})

    print(f"✅ 資料讀取完成！")
    print(f"   - 交易紀錄: {df_txn.shape[0]} 筆")
    print(f"   - 警示帳戶: {df_alert.shape[0]} 筆")
    print(f"   - 待預測帳戶: {df_predict.shape[0]} 筆")

except FileNotFoundError as e:
    print(f"❌ 錯誤：找不到檔案，請確認路徑是否正確。\n詳細錯誤: {e}")
    # 為了讓下面的 cell 不報錯，定義空的 DataFrame (僅在出錯時)
    df_txn = pd.DataFrame()
    df_alert = pd.DataFrame()
    df_predict = pd.DataFrame()

⏳ 正在讀取資料...
✅ 資料讀取完成！
   - 交易紀錄: 4435890 筆
   - 警示帳戶: 1004 筆
   - 待預測帳戶: 4780 筆


## 三、特徵工程

In [3]:
# --- 🛠️ Step 3: 特徵工程 (Feature Engineering) ---
if not df_txn.empty:
    print("⚙️ 開始計算 V4 官方特徵...")

    # 1. 處理玉山帳戶標記
    df_txn['is_esun_from'] = (df_txn['from_acct_type'] == '01').astype(int)
    df_txn['is_esun_to'] = (df_txn['to_acct_type'] == '01').astype(int)

    # --- 付款方特徵 ---
    print("   計算付款方特徵...")
    debit_features = df_txn.groupby('from_acct').agg(
        total_send_amt=('txn_amt', 'sum'),
        max_send_amt=('txn_amt', 'max'),
        min_send_amt=('txn_amt', 'min'),
        avg_send_amt=('txn_amt', 'mean'),
        count_send_txns=('txn_amt', 'count'),
        is_esun_from=('is_esun_from', 'max')
    ).reset_index().rename(columns={'from_acct': 'acct'})

    # --- 收款方特徵 ---
    print("   計算收款方特徵...")
    credit_features = df_txn.groupby('to_acct').agg(
        total_recv_amt=('txn_amt', 'sum'),
        max_recv_amt=('txn_amt', 'max'),
        min_recv_amt=('txn_amt', 'min'),
        avg_recv_amt=('txn_amt', 'mean'),
        count_recv_txns=('txn_amt', 'count'),
        is_esun_to=('is_esun_to', 'max')
    ).reset_index().rename(columns={'to_acct': 'acct'})

    # --- 合併 ---
    print("   合併特徵...")
    df_features = pd.merge(debit_features, credit_features, on='acct', how='outer')

    # 填補 NaN
    fillna_cols = [
        'total_send_amt', 'max_send_amt', 'min_send_amt', 'avg_send_amt', 'count_send_txns', 'is_esun_from',
        'total_recv_amt', 'max_recv_amt', 'min_recv_amt', 'avg_recv_amt', 'count_recv_txns', 'is_esun_to'
    ]
    df_features[fillna_cols] = df_features[fillna_cols].fillna(0)
    df_features['is_esun'] = df_features[['is_esun_from', 'is_esun_to']].max(axis=1)

    print(f"✅ 特徵計算完成！共 {df_features.shape[0]} 個帳戶。")

⚙️ 開始計算 V4 官方特徵...
   計算付款方特徵...
   計算收款方特徵...
   合併特徵...
✅ 特徵計算完成！共 1800106 個帳戶。


In [4]:
# --- 💾 Step 4 & 5 (Revised): Combine and Save a Single File ---
print("💾 Combining and saving a single feature file...")

# Label the entire feature set first
df_features_labeled = pd.merge(df_features, df_alert, on='acct', how='left')
# Create the 'label' column: 1 if in alert, 0 otherwise
# We can infer this from the non-NaN 'event_date' for alert accounts
df_features_labeled['label'] = (~df_features_labeled['event_date'].isnull()).astype(int)

# The set of accounts to be predicted
predict_acct_set = set(df_predict['acct_test'])

# Ensure all accounts from the prediction list are in the main dataframe
missing_in_features = predict_acct_set - set(df_features_labeled['acct'])
if missing_in_features:
    print(f"⚠️ Adding {len(missing_in_features)} accounts from prediction list that had no transactions.")
    missing_df = pd.DataFrame({'acct': list(missing_in_features)})
    # Use concat instead of merge for safety with dtypes
    df_features_labeled = pd.concat([df_features_labeled, missing_df], ignore_index=True).fillna(0)
    # Re-apply label after filling NaNs
    df_features_labeled['label'] = df_features_labeled['label'].astype(int)

# Now, df_features_labeled contains ALL accounts and their features/labels.
# Drop helper columns before saving.
df_to_save = df_features_labeled.drop(columns=['event_date'], errors='ignore')

print(f"Combined DataFrame shape: {df_to_save.shape}")

# Save the unified file, which will serve as the input for our PU learning script
# We name it 'preprocessing_T1_basic.csv' to match the user's expectation
output_path = os.path.join(output_dir, 'preprocessing_T1_basic.csv')
df_to_save.to_csv(output_path, index=False)

print(f"✅ Saved combined data to {output_path}")
print("🎉 All steps executed successfully!")

💾 Combining and saving a single feature file...
Combined DataFrame shape: (1800106, 15)
✅ Saved combined data to ./preprocessing_T1_basic.csv
🎉 All steps executed successfully!
